In [4]:
%load_ext autoreload
%autoreload
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display
import scipy.stats as stats
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from rdkit import Chem

epoch_num = 2

properties = ['n_atoms', 'SA', 'det. V (km/s)', 'det. P (km/s)', 'density (g/cm^3)', 'Oxygen balance 100']
#dir_path = 'energetics_test4_det_vel/'
#dir_path = 'energetics_test8/'
dir_path = ''

def count_natoms_in_smiles(s):
    try:
        natoms = Chem.MolFromSmiles(s).GetNumAtoms() #added by D.C. Elton, checks valence bonding is correct
    except:
        print("ERROR for ", s)
        natoms = 0
    return natoms

data = pd.read_csv(dir_path+str(epoch_num)+'_verified_mols.csv')

data['n_atoms'] = data['smiles'].apply(count_natoms_in_smiles)
data['Mols'] = data['smiles'].apply(Chem.MolFromSmiles)
       


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Remove duplicates 


In [5]:
data.sort_values('det. P (km/s)', ascending=False, inplace=True)
data.drop_duplicates('smiles', keep='first', inplace=True)

data.columns

Index(['Unnamed: 0', 'smiles', 'SA', 'det. V (km/s)', 'det. P (km/s)',
       'density (g/cm^3)', 'Oxygen balance 100', 'n_atoms', 'Mols'],
      dtype='object')

## create images and put in spreadsheet

In [6]:
import cairosvg
from rdkit.Chem import Draw
from rdkit.Chem.Draw import DrawingOptions
from shutil import copyfile



data['image'] = ['']*len(data)
num2print = 30
data2write = data[['det. V (km/s)', 'det. P (km/s)', 'density (g/cm^3)', 'SA', 'Oxygen balance 100']].iloc[0:num2print,:]


DrawingOptions.atomLabelFontSize = 45
DrawingOptions.dotsPerAngstrom = 100
DrawingOptions.bondLineWidth = 3.0
 

writer = pd.ExcelWriter('example_filtered_GAN_output.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
data2write.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

worksheet.set_default_row(110*2)
worksheet.set_column(0, 0, 45)
worksheet.set_row(0, 20)


mol_list = list(data['Mols'])[0:num2print]
#for (i,mol) in enumerate():

for (i, mol) in enumerate(mol_list):
    try: 
        Draw.MolToFile( mol, "temp.svg" )
        cairosvg.svg2png( url='./temp.svg', write_to='mol'+str(i+2)+'.png')
    except: 
        print("trouble printing to svg for ", i)
        copyfile('error.png', 'mol'+str(i+2)+'.png')
    
for (i, mol) in enumerate(mol_list): 
    worksheet.insert_image('A'+str(i+2), 'mol'+str(i+2)+'.png', {'x_offset': 0, 'y_offset': 0, 'x_scale': 0.75, 'y_scale':0.75})

writer.save()


# clean up folder
import glob
import os
filelist=glob.glob("mol*.png")
for file in filelist:
  os.remove(file)
os.remove("temp.svg")

